In [103]:
# IMPORTANT CONDITION:
# Attributes of each node are the attributes that have more than one value in the current DataFarme.
# Because: they might be all the same but have different classifiers, hence producing bad splits
# 
# We shall not try to find a good split of a node that has the same classifier!!!
import pandas as pd
path='data.csv'
df=pd.read_csv(path)
df=df[:20]
A=list(df.columns[:-1])
classifier=df.columns[-1]
D = df
default = 0
tree= createNode(D, A, "Root", 0, classifier)
continueTree(tree)

Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0
Generalization Error = 0.0


In [93]:
tree

{'tree':     Sex  Pclass  Embarked  Survived
 0     1       3         2         0
 1     0       1         0         1
 2     0       3         2         1
 3     0       1         2         1
 4     1       3         2         0
 5     1       3         1         0
 6     1       1         2         0
 7     1       3         2         0
 8     0       3         2         1
 9     0       2         0         1
 10    0       3         2         1
 11    0       1         2         1
 12    1       3         2         0
 13    1       3         2         0
 14    0       3         2         0
 15    0       2         2         1
 16    1       3         1         0
 17    1       2         2         1
 18    0       3         2         0
 19    0       3         0         1,
 'attributes': ['Sex', 'Pclass', 'Embarked'],
 'type': 'Root',
 'level': 0,
 'classifier': 'Survived',
 'min': 5,
 'default': 1,
 'gini': 0.5,
 'feature': ('Sex', 1),
 'left': {'tree':     Sex  Pclass  Embarked  Su

# FUNCTIONS

In [94]:
d=tree["tree"]
ds=d[d["Sex"]==1]
dd=d[d["Sex"]==0]
getVals(d, "Sex")

ddd=dd[dd["Pclass"]<3]
dds=dd[dd["Pclass"]>=3]
ddsd=dds[dds["Embarked"]<2]
ddss=dds[dds["Embarked"]>=2]

dsd=ds[ds["Pclass"]<3]
dss=ds[ds["Pclass"]>=3]

a=ddss
GINI(a, "Survived")
a

,Sex,Pclass,Embarked,Survived
2,0,3,2,1
8,0,3,2,1
10,0,3,2,1
14,0,3,2,0
18,0,3,2,0


In [95]:
atts=["Sex", "Pclass", "Embarked"]
d=ddss
ginis={}
for att in atts:
    vals = getVals(d, att)
    print(vals)
    for val in vals[1:]:
        ginis[(att ,val)]=GINIsplit(d, att, val, "Survived")
best = min(ginis, key=ginis.get)
best
ginis

[0]
[3]
[2]


ValueError: min() arg is an empty sequence

In [96]:
a,b=split(dds, "Pclass", 2)
a

,Sex,Pclass,Embarked,Survived


In [97]:
bestCandidate(dds, atts, "Survived")
GINI(ddss,"Survived")

0.48

In [98]:
allEqual(ddss, tree["attributes"])

True

In [99]:
a =[]
a ==[]

True

In [100]:
generalizationError(tree, 0.5)

Generalization Error = 0.275


(5.5, 3, 5)

In [135]:
pruneTree(tree, 0.5)

Generalization Error = 0.20192307692307693
Generalization Error = 0.2001733102253033
Generalization Error = 0.06845238095238096
Generalization Error = 0.3576388888888889
Generalization Error = 0.38934426229508196
Generalization Error = 0.14945054945054945


0

In [134]:
def pruneTree(tree, alpha):
    prunedTree=dict(tree)
    for level in nodesByLevel[1:getLevels(tree)[-1]]:
        for node in level:
            generalizationError(node, alpha)
    return 0

In [115]:
def getLevels(tree):
    nodesByLevel= getTreeOrdered(tree)
    levels =[0]
    for a in nodesByLevel:
        for val in a:
            level= val["level"]
            if level not in levels:
                levels += [level]
    return levels

In [116]:
def generalizationError(tree, alpha):
    nodesByLevel= getTreeOrdered(tree)
    complexity=0
    mistakes=0
    for level in nodesByLevel[1:]:
        for node in level:
            if(node["type"]=="Leaf"):
                complexity+=1
                vals=node["tree"].T
                for row in node["tree"].T:
                    test = sum(node["tree"][node["classifier"]])/len(node["tree"]) > 0.5
                    cl = 1 if test else 0
                    if (len(node["tree"])<node["min"]): cl = node["default"]
                    if(vals[row][node["classifier"]]!=cl):
                        mistakes+=1
    error = mistakes + alpha * complexity
    print("Generalization Error =",error/len(tree["tree"]))
    return error, mistakes, complexity

In [117]:
def printDecisionTree(tree):
    printNode(tree)
    print()
    nodesByLevel= getTreeOrdered(tree)
    for level in nodesByLevel[1:]:
        if level == []:
            continue
        i=0
        for node in level:
            i+=1
            printNode(node)
            if (i != len(level)):
                print('*********')
        print()

In [118]:
def getTreeOrdered(tree):
    nodesByLevel=[[]]*len(tree["tree"].columns)*4 #recently edited!!!
    nodesByLevel[0] = [tree]
    getNextLevel(tree, nodesByLevel)
    return nodesByLevel

In [119]:
def getNextLevel(tree, result):
    if 'left' in tree.keys():
        result[tree['level']+1]= result[tree['level']+1] + [tree['left']]
        result[tree['level']+1]= result[tree['level']+1] + [tree['right']]
        getNextLevel(tree['left'],result)
        getNextLevel(tree['right'],result)

In [120]:
def printNode(node):
    nType = node["type"]
    nLevel= node["level"]
    nGini =node["gini"]
    print(nType)
    print("Level",nLevel)
    if nType in ("Root","Intermediate"):
        feature = node["feature"]
        print("Feature",feature[0],end=" ")
        for i in range(min(getVals(node["tree"],feature[0])),feature[1]):
            if (i != feature[1] - 1):
                print(i, end = " ")
            else:
                print(i, end = "")
        print()
    else:
        test = sum(node["tree"][node["classifier"]])/len(node["tree"]) > 0.5
        cl = 1 if test else 0
        if (len(node["tree"])<node["min"]): cl = node["default"]
        print("Class",cl)
    if(nGini!=0):
        print("Gini","{:.4}".format(nGini))
    else:
        print("Gini",0)

In [121]:
def continueTree(root):
    A = list(root["attributes"])
    D = root["tree"]
    classifier = root["classifier"]
    sameAttributes = allEqual(D, A)
    if len(D)<=root["min"] or root["gini"]==0 or sameAttributes:
        root["type"]= "Leaf"
    elif root["type"] != "Leaf":
        root["feature"]= [attribute, value] = bestCandidate(D, A, classifier) #don't call for best candidate if there are no new best candidates
        left,right = split(D, attribute, value)
        root["left"]  = createNode(left , A, "Intermediate", root["level"]+1, classifier)
        root["right"] = createNode(right, A, "Intermediate", root["level"]+1, classifier)
        continueTree(root["left"])
        continueTree(root["right"])

In [122]:
def allEqual(D, A):
    for att in A:
        iterator = D[att]
        iterator = iter(iterator)
        try:
            first = next(iterator)
        except StopIteration:
            return True
        answer = all(first == rest for rest in iterator)
        if answer == False:
            return answer
    return True

In [123]:
def bestCandidate(data, attributes, classifier):
    ginis={}
    for att in atts:
        vals = getVals(data, att)
        for val in vals[1:]:
            ginis[(att ,val)]=GINIsplit(data, att, val, classifier)
    best = min(ginis, key=ginis.get)
    return best

In [124]:
def GINIsplit(data, attribute, value, classifier):
    """returns the Gini of the split if applied on the given attribute with the specified classifier, input is a dataframe"""
    [yes,no]=split(data, attribute, value)
    gini = (GINI(yes,classifier)*len(yes)+GINI(no, classifier)*len(no))/len(data)
    return gini

In [125]:
def split(data, attribute, value):
    """returns two nodes based on a given split (attribute < value), input is a dataframe"""
    yes = data[data[attribute] < value]
    no = data[data[attribute] >= value]
    return yes,no

In [126]:
def getVals(df, attribute):
    """returns the DISTINCT values of a specified attribute from a dataset"""
    vals = []
    for val in list(df[attribute]):
        if val not in vals:
            vals += [val]
    vals.sort()
    return vals

In [127]:
def GINI(data, classifier):
    """returns the Gini of the given node, based on the Class"""
    classVals=getVals(data,classifier)
    cl=data[classifier]
    sqSum = 0
    for val in classVals:
        prob= len(cl[cl==val])/len(data)
        sqSum += prob**2
    GINI= 1-sqSum
    return GINI

In [128]:
def createNode(D, A, nodeType, level, classifier, minNum=5,alpha=0.5, default = 1):
    node = {}
    node["tree"] = D
    node["attributes"] = A
    node["type"] = nodeType
    node["level"] = level
    node["classifier"] = classifier
    node["min"]=minNum
    node["default"]=default
    node["gini"]=float(GINI(D,classifier))
    return node